# python is weird

a meditation on the flaming pile of weirdness that is python, hosted by josh, with content heavily borrowed from [wtfpython](https://github.com/satwikkansal/wtfpython).

# intro

In [ ]:
a = 1

In [ ]:
2 is 2

In [ ]:
2 is 2.0

# 1

In [ ]:
a = 'hello'
b = 'hello'

In [ ]:
a == b

In [ ]:
a is b

In [ ]:
a = 'hello!'
b = 'hello!'

In [ ]:
a == b

In [ ]:
a is b

In [ ]:
a = '!'
b = '!'

In [ ]:
a == b

In [ ]:
a is b

<details>
  <summary>what's going on?</summary>
  
- CPython optimization (called string interning) tries to use existing immutable objects in some cases rather than creating a new object every time.
- After being "interned," many variables may reference the same string object in memory (saving memory thereby).
- In the snippets above, strings are implicitly interned. The decision of when to implicitly intern a string is implementation-dependent. There are some rules that can be used to guess if a string will be interned or not:
    - All length 0 and length 1 strings are interned.
    - Strings are interned at compile time ('wtf' will be interned but ''.join(['w', 't', 'f']) will not be interned)
    - Strings that are not composed of ASCII letters, digits or underscores, are not interned. This explains why 'wtf!' was not interned due to !.
</details>

# 2

In [ ]:
(False == False) in [False]

In [ ]:
False == (False in [False])

In [ ]:
False == False in [False]

In [ ]:
True is False == False

In [ ]:
False is False is False

In [ ]:
(1 > 0) < 1

In [ ]:
1 > (0 < 1)

In [ ]:
1 > 0 < 1

<details>
  <summary>what's going on?</summary>
  
>Formally, if a, b, c, ..., y, z are expressions and op1, op2, ..., opN are comparison operators, then a op1 b op2 c ... y opN z is equivalent to a op1 b and b op2 c and ... y opN z, except that each expression is evaluated at most once.
</details>

# 3

In [ ]:
a = 256
b = 256

In [ ]:
a == b

In [ ]:
a is b

In [ ]:
a = 257
b = 257

In [ ]:
a == b

In [ ]:
a is b

In [ ]:
a, b = 257, 257

In [ ]:
a == b

In [ ]:
a is b

In [ ]:
a = []
b = []

In [ ]:
a == b

In [ ]:
a is b

In [ ]:
a = tuple()
b = tuple()

In [ ]:
a == b

In [ ]:
a is b

<details>
  <summary>what's going on?</summary>
    
- <code>256</code> is an existing object but <code>257</code> isn't. When you start up python the numbers from <code>-5</code> to <code>256</code> will be allocated. These numbers are used a lot, so it makes sense just to have them ready.   
- The interpreter isn't smart enough while executing <code>b = 257</code> to recognize that we've already created an integer of the value <code>257</code>, and so it goes on to create another object in the memory.
- Both <code>a</code> and <code>b</code> refer to the same object when initialized with same value in the same line.
- Similar optimization applies to other immutable objects like empty tuples as well. Since lists are mutable, that's why [] is [] will return False and () is () will return True.
</details>

# 4

In [ ]:
some_dict = {}
some_dict[5.5] = "josh"
some_dict[5.0] = "is"
some_dict[5] = "cool"

In [ ]:
some_dict[5.5] == 'josh'

In [ ]:
some_dict[5.0] == 'is'

In [ ]:
some_dict[5] == 'cool'

In [ ]:
complex_five = 5 + 0j

In [ ]:
type(complex_five) == complex

In [ ]:
some_dict[complex_five] == 'cool'

<details>
  <summary>what's going on?</summary>
    
- Uniqueness of keys in a Python dictionary is by equivalence, not identity. So even though <code>5</code>, <code>5.0</code>, and <code>5 + 0j</code> are distinct objects of different types, since they're equal, they can't both be in the same dict (or set). As soon as you insert any one of them, attempting to look up any distinct but equivalent key will succeed with the original mapped value (rather than failing with a <code>KeyError</code>)
- This applies when setting an item as well. So when you do <code>some_dict[5] = "cool"</code>, Python finds the existing item with equivalent key <code>5.0 -> "cool"</code>, overwrites its value in place, and leaves the original key alone.
- So how can we update the key to <code>5</code> (instead of <code>5.0</code>)? We can't actually do this update in place, but what we can do is first delete the key (<code>del some_dict[5.0]</code>), and then set it (<code>some_dict[5]</code>) to get the integer <code>5</code> as the key instead of floating <code>5.0</code>, though this should be needed in rare cases.
- How did Python find <code>5</code> in a dictionary containing <code>5.0</code>? Python does this in constant time without having to scan through every item by using hash functions. When Python looks up a key foo in a dict, it first computes <code>hash(foo)</code> (which runs in constant-time). Since in Python it is required that objects that compare equal also have the same hash value (docs here), <code>5</code>, <code>5.0</code>, and <code>5 + 0j</code> have the same hash value.
</details>

# 5

In [ ]:
row = [""] * 3
row

In [ ]:
board = [row] * 3
board

In [ ]:
board[0]

In [ ]:
board[0][0]

In [ ]:
board[0][0] = "X"

In [ ]:
board

<details>
  <summary>what's going on?</summary>
    
When the board is initialized by multiplying the row, each of the elements <code>board[0]</code>, <code>board[1]</code> and <code>board[2]</code> is a reference to the same list referred by <code>row</code>.
</details>

In [ ]:
board = [[''] * 3 for _ in range(3)]
board[0][0] = "X"

In [ ]:
board

# 6

In [ ]:
a, b = a[b] = {}, 5

In [ ]:
a

<details>
  <summary>what's going on?</summary>
    fuck knows. <a href="https://github.com/satwikkansal/wtfpython#-lets-see-if-you-can-guess-this">this</a>.
    </details>
</details>


# 7

In [ ]:
x = 7, 8, 9

In [ ]:
sorted(x) == x

In [ ]:
sorted(x) == sorted(x)

<details>
  <summary>what's going on?</summary>
    <code>x</code> is a tuple. <code>sorted(x)</code> returns a list. Comparing lists and tuples always returns <code>False</code> in python.
</details>


# 8

In [ ]:
import antigravity

# 9

In [ ]:
...

<details>
  <summary>what's going on?</summary>
    It's a thing go look it up.

</details>


# 10

In [ ]:
[] = []

In [ ]:
() = ()

In [ ]:
{} = {}

# 11

In [ ]:
'a'[0][0][0]

In [ ]:
'a'[0][1][0]

In [ ]:
'a'[0][0][-1]

<details>
  <summary>what's going on?</summary>
    <code>'a'</code> is a string, and indexing a string gets you a substring, or single character. So <code>'a'[0]</code> returns <code>'a'</code>.

</details>


# 12

In [ ]:
a = 5

In [ ]:
++a

In [ ]:
a++

<details>
  <summary>what's going on?</summary>
    <code>++a</code> is the same as <code>+(+a)</code>. <code>a++</code> is invalid.

</details>


# 13

In [ ]:
a = 5

In [ ]:
a -=- 1

In [ ]:
a

In [ ]:
a +=+ 1

In [ ]:
a

In [ ]:
a -= (-1)

<details>
  <summary>what's going on?</summary>
    <code>a -=- 1</code> is deceptive because that last minus symbol attaches itself to the 1, so it's really <code>a -= (-1)</code>. Same with <code>a +=+ 1</code>.

</details>
